we have assume the relationship bewteen cashierpayment,cashier_final，cashierdetail table. So we need to validate our assumption her

将cashierpayment和cashier_final，cashierdetail分别保存到df中，在cashier的部分，我们已经处理了noTrans值等于01SCS19E310345的部分，并且将ashierpayment的noTrans值等于01SCS19E310345删除，但是根据外键的关联性，在cashierdetail部分同时存在noTrans值等于01SCS19E310345的部分，所以首先将其删除。

In [1]:
# combine three csv cashierpayment,cashier_final，cashierdetail
import pandas as pd
df1 = pd.read_csv(r'format_data\cashier_final.csv', delimiter=',', encoding='utf-8')
df2 = pd.read_csv(r'format_data\cashierpayment_final.csv', delimiter=',', encoding='utf-8')
df3 = pd.read_csv(r'format_data\cashierdetail_final.csv', delimiter=',', encoding='utf-8')

In [2]:
# 查看在cashierdetail中noTrans值等于01SCS19E310345的部分
df3[df3['noTrans'] == '01SCS19E310345']

,noTrans,ArticleCode,Barcode,sizes,qty,basePrice,salePrice,DiscountType,discountPersen,discountRupiah,DiscExpenses,consignment,consignmentRp,subTotal,payment
11307,01SCS19E310345,AR.PCT-SE005,20ARSE0344,M,1,87750,135000,2,0,27000,1,0,27000,108000,81000


In [3]:
# 清除noTrans值等于01SCS19E310345的部分
df3 = df3[df3['noTrans'] != '01SCS19E310345']
# 再次查看
df3[df3['noTrans'] == '01SCS19E310345']

,noTrans,ArticleCode,Barcode,sizes,qty,basePrice,salePrice,DiscountType,discountPersen,discountRupiah,DiscExpenses,consignment,consignmentRp,subTotal,payment


接下来查看notrans列还有什么值不同。将三个df中的notrans列作为列表存储

In [4]:
ca_no1 = df1['noTrans'].tolist()
ca_no2 = df2['noTrans'].tolist()
ca_no3 = df3['noTrans'].tolist()

## 首先看cashier 和cashier_payment列不同

In [5]:
# 查找df1和df2的不同值
not_intersects_1 = list(set(ca_no1).symmetric_difference(set(ca_no2)))
not_intersects_1

['01SCS19E070078', '01SCS19E310345', '01SCS19E310181']

In [6]:
# 查找df1和df3不同值
not_intersects_2 = list(set(ca_no1).symmetric_difference(set(ca_no3)))
not_intersects_2

['01SCS19E310345']

In [7]:
# df2和df3的不同值
not_intersects_3 = list(set(ca_no2).symmetric_difference(set(ca_no3)))
not_intersects_3

['01SCS19E070078', '01SCS19E310181']

根据上面的处理，可以得到在cashierpayment中缺失了'01SCS19E310181', '01SCS19E070078'，'01SCS19E010027'这三个notrans值，而cashier里缺失了01SCS19E010027这个值。根据erd图，payment中的信息我无法根据现有的数据补全（这里随便编），所以在这里，我将这三列从这三个df中删除以保证他们的外键数量一致。

In [8]:
df1[(df1['noTrans'] == '01SCS19E310181') | (df1['noTrans'] == '01SCS19E070078')]

,noTrans,dateTrans,typeTrans,Notes,UserID,referenceTrans
1150,01SCS19E070078,2019-05-07 16:35:16.000,CASHIER,NaN,CHANDRA,NaN
7640,01SCS19E310181,2019-05-31 15:54:58.000,CASHIER,NaN,LISDA,NaN


In [9]:
df1 = df1[(df1['noTrans'] != '01SCS19E310181') & (df1['noTrans'] != '01SCS19E070078') & (df1['noTrans'] != '01SCS19E010027')]
# 再次查看
df1[(df1['noTrans'] == '01SCS19E310181') | (df1['noTrans'] == '01SCS19E070078')|(df1['noTrans'] == '01SCS19E010027')]


,noTrans,dateTrans,typeTrans,Notes,UserID,referenceTrans


cashier清理完毕。接着处理cashierdetail。将多余的两列处理掉

In [10]:
df3 = df3[(df3['noTrans'] != '01SCS19E310181') & (df3['noTrans'] != '01SCS19E070078')]
df3[(df3['noTrans'] == '01SCS19E310181') | (df3['noTrans'] == '01SCS19E070078')]

,noTrans,ArticleCode,Barcode,sizes,qty,basePrice,salePrice,DiscountType,discountPersen,discountRupiah,DiscExpenses,consignment,consignmentRp,subTotal,payment


最后把cashierpayment中的01SCS19E010027列删除

In [11]:
df2[(df2['noTrans'] == '01SCS19E010027')]
df2 = df2[(df2['noTrans'] != '01SCS19E010027')]

处理完成后重新查看

In [12]:
ca_no1_clean = df1['noTrans'].tolist()
ca_no2_clean = df2['noTrans'].tolist()
ca_no3_clean = df3['noTrans'].tolist()

not_intersects_1_clean = list(set(ca_no1_clean).symmetric_difference(set(ca_no1_clean)))
not_intersects_2_clean = list(set(ca_no1_clean).symmetric_difference(set(ca_no3_clean)))
not_intersects_3_clean = list(set(ca_no2_clean).symmetric_difference(set(ca_no3_clean)))

In [13]:
not_intersects_1_clean

[]

In [14]:
not_intersects_2_clean

['01SCS19E010027', '01SCS19E310345']

In [15]:
not_intersects_3_clean

['01SCS19E010027']

没有不一样的值，说明这三个df的外键已经完全统一。将数据保存成csv

In [ ]:
df1.to_csv('format_data/cashier_final.csv', index=False)
df2.to_csv('format_data/cashierpayment_final.csv', index=False)
df3.to_csv('format_data/cashierdetail_final.csv', index=False)